# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

global left_previous
global right_previous

left_previous = []
right_previous = []

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def find_endpts(lines):
    
    lowest = [lines[0][0][0], lines[0][0][1]]
    highest = [lines[0][0][2], lines[0][0][3]]
    
    for line in lines:
        if line[0][1] >= lowest[1]:
            lowest = [line[0][0], line[0][1]]
        if line[0][3] <= highest[1]:
            highest = [line[0][2], line[0][3]]
    return [lowest, highest]


def lane_line(x1, y1, x2, y2):
    
    mn = (y2-y1)/(x2-x1)
    bn = y1 - (mn * x1)
    
    
    """
    mp = (previous[3]-previous[1])/(previous[2]-previous[0])
    bp = previous[1] - (mp * previous[0])
    
    if (mp/mn > 1.4 or mp/mn < 0.6 or bn > bp*1.4 or bn < bp * 0.6):
        m = mp
        b = bp
    elif (mp/mn > 1.3 or mp/mn < 0.7):
        m = mp*0.7 + mn*0.3
        b = bn
    else:
        m = (mp + mn)/2
        b = (bp + bn)/2
    """
    m = mn
    b = bn
    v1 = 720 #height
    v2 = 380 #center_height
    
    u1 = int((v1 - b) / m)
    u2 = int((v2 - b) / m)
    
    return [u1, v1, u2, v2]


def create_original(x1, y1, x2, y2):
    m = (y2-y1)/(x2-x1)
    b = (y2 - (m*x2))
    
    v1 = 719
    v2 = 380 #center_height
    
    u1 = int((v1-b)/m)
    u2 = int((v2-b)/m)
    
    return [u1, v1, u2, v2]


def valid_slope(ends):
    if (ends[0][0] == ends[1][0]) or (ends[0][1] == ends[1][1]):
        return False
    else:
        return True
    
def get_slope(lst):
    return (((lst[3]-lst[1])/(lst[2]-lst[0]))*180/np.pi)

def median_slope(lines):
    slopes = []
    for line in lines:
        slopes.append(((line[3]-line[1])/(line[2]-line[0]))*180/np.pi)
    return np.median(slopes)


## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
#os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
# image = mpimg.imread('test_images/solidYellowCurve2.jpg')

from moviepy.editor import VideoFileClip
from moviepy.video.VideoClip import VideoClip
from IPython.display import HTML

# clip1 = VideoFileClip("challenge.mp4")
# clip1.save_frame('test2.jpg', t='00:00:03')

image = mpimg.imread('test.jpg')

global width
width = image.shape[1]
global height
height = image.shape[0]
global center_height
center_height = int(height * 0.5926)
global left_bottom
left_bottom = int(width * 0.13925)
global right_bottom
right_bottom = int(width - (width * 0.0625))

left_center = int((width/2) - (width * 0.0408))
right_center = int((width/2) + (width * 0.0521))

blue = image[:,:,0]
blurred = gaussian_blur(blue, 7)
cannied = canny(blurred, 80, 200)
vertices = np.array([[[left_bottom, height], [left_center, center_height], [right_center, center_height], [right_bottom, height]]])
cannied_region = region_of_interest(cannied, vertices)

line_img = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
#cv2.fillPoly(line_img, vertices, [100,0,0])
houghlines = cv2.HoughLinesP(cannied_region, 0.5, np.pi/360, 4, 30, 5)

houghlines = np.squeeze(houghlines)

left = []
right = []

if len(houghlines) > 1:
    for line in houghlines:
        if (line[3]-line[1])/(line[2]-line[0]) < 0:
            left.append(line)
            #cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), [0,0,200], 8)
        elif (line[3]-line[1])/(line[2]-line[0]) > 0:
            right.append(line)
            #cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), [0,0,200], 8)

slope_left = median_slope(left)
slope_right = median_slope(right)

new_left = []
new_right = []

for line in left:
    if (abs(get_slope(line) - slope_left) < 8):
        new_left.append(line)
        
for line in right:
    if (abs(get_slope(line) - slope_right) < 8):
        new_right.append(line)

# Now lets use cv2.fitline

left_points = []
for line in new_left:
    left_points.append((line[0], line[1]))
    left_points.append((line[2], line[3]))

right_points = []
for line in new_right:
    right_points.append((line[0], line[1]))
    right_points.append((line[2], line[3]))

[vxl, vyl, xl, yl] = cv2.fitLine(np.array(left_points, dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
[vxr, vyr, xr, yr] = cv2.fitLine(np.array(right_points, dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)

#Get current slopes
ml = vyl/vxl
mr = vyr/vxr

#Compare with previous frame
if len(left_previous) == 2:
    [m_old, b_old] = left_previous
    if (abs((ml*180/np.pi) - m_old) > 10):
        ml = (m_old * np.pi/180)*0.8 + ml*0.2
    if (abs((ml*180/np.pi) - m_old) > 5):
        ml = (m_old * np.pi/180)*0.7 + ml*0.3
    else:
        ml = (m_old * np.pi/180)*0.5 + ml*0.5

if len(right_previous) == 2:
    [m_old, b_old] = right_previous
    if (abs((mr*180/np.pi) - m_old) > 10):
        mr = (m_old * np.pi/180)*0.8 + mr*0.2
    if (abs((mr*180/np.pi) - m_old) > 5):
        mr = (m_old * np.pi/180)*0.7 + mr*0.3
    else:
        mr = (m_old * np.pi/180)*0.5 + mr*0.5

# Get current intercept using new slopes
bl = (yl - ml*xl)
br = (yr - mr*xr)

# Set new REFERENCE
left_previous = [ml*180/np.pi, bl]
right_previous = [mr*180/np.pi, br]

#Draw the lines
x1l = int((height - bl)/ml)
x2l = int((center_height - bl)/ml)
cv2.circle(line_img, (xl,yl), 10, [255,0,0], 3)
cv2.line(line_img, (x1l, height), (x2l, center_height), [255,0,0], 5)

x1r = int((height - br)/mr)
x2r = int((center_height - br)/mr)
cv2.circle(line_img, (xr,yr), 10, [255,0,0], 3)
cv2.line(line_img, (x1r, height), (x2r, center_height), [255,0,0], 5)

"""        
    if len(left) < 1:
        pass
    else:
        left_ends = find_endpts(left)

        if len(left_previous) == 4 and valid_slope(left_ends): # i.e reference exists
            left_lane = lane_line(left_ends[0][0], left_ends[0][1], left_ends[1][0], left_ends[1][1], left_previous)
            left_previous = left_lane
            cv2.line(line_img, (left_lane[0], left_lane[1]), (left_lane[2], left_lane[3]), [59,165,46], 5)
            
        else:
            if valid_slope(left_ends):
                left_lane = create_original(left_ends[0][0], left_ends[0][1], left_ends[1][0], left_ends[1][1])
                left_previous = left_lane
                cv2.line(line_img, (left_lane[0], left_lane[1]), (left_lane[2], left_lane[3]), [59,165,46], 5)
            else:
                pass
            
    if len(right) < 1:
        pass
    else:
        right_ends = find_endpts(right)
        
        if len(right_previous) == 4 and valid_slope(right_ends): # i.e reference exists
            right_lane = lane_line(right_ends[0][0], right_ends[0][1], right_ends[1][0], right_ends[1][1], right_previous)
            right_previous = right_lane
            cv2.line(line_img, (right_lane[0], right_lane[1]), (right_lane[2], right_lane[3]), [59,165,46], 5)
        else:
            if valid_slope(right_ends):
                right_lane = create_original(right_ends[0][0], right_ends[0][1], right_ends[1][0], right_ends[1][1])
                right_previous = right_lane
                cv2.line(line_img, (right_lane[0], right_lane[1]), (right_lane[2], right_lane[3]), [59,165,46], 5)
            else:
                pass
                
    #cv2.line(line_img, (left_ends[0][0], left_ends[0][1]), (left_ends[1][0], left_ends[1][1]), [59,165,46], 5)
    #plt.imshow(line_img)

    #m = (left_ends[1][1]-left_ends[0][1])/(left_ends[1][0]-left_ends[0][0])
    #b = (left_ends[0][1] - (m*left_ends[0][0]))

    # print (((left_ends[0][1] - b)/m), height)
    # print (((left_ends[1][1] - b)/m), center_height)

else:
    pass
"""
overlay = weighted_img(line_img, image)

plt.imshow(overlay)


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.video.VideoClip import VideoClip
from IPython.display import HTML

global width
global height
global center_height
global left_bottom
global right_bottom
global left_previous
global right_previous

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    width = image.shape[1]
    height = image.shape[0]
    center_height = int(height * 0.5926)
    left_bottom = int(width * 0.10625)
    right_bottom = int(width - (width * 0.0625))
    left_center = int((width/2) - (width * 0.0108))
    right_center = int((width/2) + (width * 0.0521))
    
    blue = image[:,:,0]
    blurred = gaussian_blur(blue, 7)
    cannied = canny(blurred, 80, 200)
    vertices = np.array([[[left_bottom, height], [left_center, center_height], [right_center, center_height], [right_bottom, height]]])
    cannied_region = region_of_interest(cannied, vertices)

    line_img = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    #cv2.fillPoly(line_img, vertices, [100,0,0])
    houghlines = cv2.HoughLinesP(cannied_region, 0.5, np.pi/360, 4, 30, 5)

    houghlines = np.squeeze(houghlines)

    left = []
    right = []

    if len(houghlines) > 1:
        for line in houghlines:
            if (line[3]-line[1])/(line[2]-line[0]) < 0:
                left.append(line)
                #cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), [0,0,200], 8)
            elif (line[3]-line[1])/(line[2]-line[0]) > 0:
                right.append(line)
                #cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), [0,0,200], 8)

    slope_left = median_slope(left)
    slope_right = median_slope(right)

    new_left = []
    new_right = []

    for line in left:
        if (abs(get_slope(line) - slope_left) < 8):
            new_left.append(line)

    for line in right:
        if (abs(get_slope(line) - slope_right) < 8):
            new_right.append(line)

    # Now lets use cv2.fitline

    left_points = []
    for line in new_left:
        left_points.append((line[0], line[1]))
        left_points.append((line[2], line[3]))

    right_points = []
    for line in new_right:
        right_points.append((line[0], line[1]))
        right_points.append((line[2], line[3]))

    [vxl, vyl, xl, yl] = cv2.fitLine(np.array(left_points, dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
    [vxr, vyr, xr, yr] = cv2.fitLine(np.array(right_points, dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)

    #Get current slopes
    ml = vyl/vxl
    mr = vyr/vxr

    #Compare with previous frame
    if len(left_previous) == 2:
        [m_old, b_old] = left_previous
        if (abs((ml*180/np.pi) - m_old) > 10):
            ml = (m_old * np.pi/180)*0.8 + ml*0.2
        if (abs((ml*180/np.pi) - m_old) > 5):
            ml = (m_old * np.pi/180)*0.7 + ml*0.3
        else:
            ml = (m_old * np.pi/180)*0.5 + ml*0.5

    if len(right_previous) == 2:
        [m_old, b_old] = right_previous
        if (abs((mr*180/np.pi) - m_old) > 10):
            mr = (m_old * np.pi/180)*0.8 + mr*0.2
        if (abs((mr*180/np.pi) - m_old) > 5):
            mr = (m_old * np.pi/180)*0.7 + mr*0.3
        else:
            mr = (m_old * np.pi/180)*0.5 + mr*0.5

    # Get current intercept using new slopes
    bl = (yl - ml*xl)
    br = (yr - mr*xr)

    # Set new REFERENCE
    left_previous = [ml*180/np.pi, bl]
    right_previous = [mr*180/np.pi, br]

    #Draw the lines
    x1l = int((height - bl)/ml)
    x2l = int((center_height - bl)/ml)
    cv2.circle(line_img, (xl,yl), 10, [255,0,0], 3)
    cv2.line(line_img, (x1l, height), (x2l, center_height), [255,0,0], 5)

    x1r = int((height - br)/mr)
    x2r = int((center_height - br)/mr)
    cv2.circle(line_img, (xr,yr), 10, [255,0,0], 3)
    cv2.line(line_img, (x1r, height), (x2r, center_height), [255,0,0], 5)
    
    overlay = weighted_img(line_img, image)
    return overlay

#plt.imshow(process_image(mpimg.imread('test.jpg')))

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))